In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input/kkbox-churn-prediction-challenge"]).decode("utf8"))
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
import numpy as np # linear alegbra
import pandas as pd # data processing
import os # os commands
from datetime import datetime as dt #work with date time format
%matplotlib notebook 
# initiate matplotlib backend
import seaborn as sns # work over matplotlib with improved and more graphs
import matplotlib.pyplot as plt #some easy plotting



In [ ]:
transactions = pd.read_csv('../input/transactions.csv', engine = 'c', sep=',')#reading the transaction file

In [ ]:
transactions_v2 =pd.read_csv('../input/transactions_v2.csv', engine = 'c', sep=',')

In [ ]:
transactions.info()

In [ ]:
transactions.describe()

** to reduce the size of transactions dataframe**

In [ ]:

print("payment_plan_days min: ",transactions['payment_plan_days'].min())
print("payment_plan_days max: ",transactions['payment_plan_days'].max())

print('payment_method_id min:', transactions['payment_method_id'].min())
print('payment_method_id max:', transactions['payment_method_id'].max())


In [ ]:
# h=change the type of these series

transactions['payment_method_id'] = transactions['payment_method_id'].astype('int8')
transactions['payment_plan_days'] = transactions['payment_plan_days'].astype('int16')

transactions_v2['payment_method_id'] = transactions_v2['payment_method_id'].astype('int8')
transactions_v2['payment_plan_days'] = transactions_v2['payment_plan_days'].astype('int16')

In [ ]:
print('plan list price varies from ', transactions['plan_list_price'].min(), 'to ',transactions['plan_list_price'].max() )
print('actual amount varies from ', transactions['actual_amount_paid'].min(),'to ', transactions['actual_amount_paid'].max() )

In [ ]:

transactions['plan_list_price'] = transactions['plan_list_price'].astype('int16')
transactions['actual_amount_paid'] = transactions['actual_amount_paid'].astype('int16')
transactions_v2['plan_list_price'] = transactions_v2['plan_list_price'].astype('int16')
transactions_v2['actual_amount_paid'] = transactions_v2['actual_amount_paid'].astype('int16')

** size of file has decreased by almost 33% **

In [ ]:
transactions.info()

In [ ]:

transactions['is_auto_renew'] = transactions['is_auto_renew'].astype('int8') # chainging the type to boolean
transactions['is_cancel'] = transactions['is_cancel'].astype('int8')#changing the type to boolean

transactions_v2['is_auto_renew'] = transactions_v2['is_auto_renew'].astype('int8') # chainging the type to boolean
transactions_v2['is_cancel'] = transactions_v2['is_cancel'].astype('int8')#changing the type to boolean

In [ ]:
sum(transactions.memory_usage()/1024**2) # memory usage 

In [ ]:

transactions['membership_expire_date'] = pd.to_datetime(transactions['membership_expire_date'].astype(str), infer_datetime_format = True, exact=False)
transactions_v2['membership_expire_date'] = pd.to_datetime(transactions_v2['membership_expire_date'].astype(str), infer_datetime_format = True, exact=False)
# converting the series to string and then to datetime format for easy manipulation of dates
sum(transactions.memory_usage()/1024**2) # this wouldn't change the size of df as memory occupied by object is similar to datetime

In [ ]:

transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'].astype(str), infer_datetime_format = True, exact=False)

transactions_v2['transaction_date'] = pd.to_datetime(transactions_v2['transaction_date'].astype(str), infer_datetime_format = True, exact=False)

** repeating the same process on members file/df**

In [ ]:
members = pd.read_csv('../input/members_v3.csv')



In [ ]:
members.info()

In [ ]:
members.describe()

In [ ]:

members['city']=members['city'].astype('int8');
members['bd'] = members['bd'].astype('int16');
members['bd']=members['bd'].astype('int8');
members['registration_init_time'] = pd.to_datetime(members['registration_init_time'].astype(str), infer_datetime_format = True, exact=False)
#members['expiration_date'] = pd.to_datetime(members['expiration_date'].astype(str), infer_datetime_format = True, exact=False)

** doing the same with train data**

In [ ]:

train = pd.read_csv('../input/train.csv')
train_v2 = pd.read_csv('../input/train_v2.csv')
train.head()
train['is_churn'] = train['is_churn'].astype('int8');
train_v2['is_churn'] = train_v2['is_churn'].astype('int8');

In [ ]:
transactions['msno'].head(10)

In [ ]:
train['msno'].nunique()#.append(train_v2)['msno'].nunique()

In [ ]:
transactions['msno'].nunique()

In [ ]:
len(transactions_v2['msno'])#.nunique()

In [ ]:
transactions[transactions['msno']=='uQxbyACsPOEkTIrv9jZgoGXelGBW81ZsSZKy9fhj5Z8='].sort_values('membership_expire_date')

** now merging all the dataframe with inner joint as we would not want half information about users**

In [ ]:

members_trans = members.merge(transactions, how='inner', on='msno')
data = members_trans.merge(train, how='inner', on='msno')
# deleting the previously imported df as they occupy space in memory
del transactions
del train
del members_trans

In [ ]:
members_trans = members.merge(transactions_v2, how='inner', on='msno')
test_data = members_trans.merge(train_v2, how='inner', on='msno')
# deleting the previously imported df as they occupy space in memory
del transactions_v2
del members
del train_v2
del members_trans

In [ ]:
#total memory consumptions by all these data frame
sum(data.memory_usage()/1024**2)

Number of values in missing in data 

In [ ]:
sum(data['gender'].isnull())/len(data)

In [ ]:
sum(test_data['gender'].isnull())/len(test_data)

~52% of gender data is missing or the users did not provide the data. This can be ascertained by seeing whether their is signinficant difference in churn of those whose gender is available vs those whose gender data is not available.

******EAD for train data and dummy variables**

In [ ]:
plt.figure()
data.groupby(['msno','gender'])['is_churn'].mean().groupby(level=1).mean().plot.bar();
#taking the mean will give us a fraction of people churning as values are 0/1

** There seems to be no correlation between geder and churn, but let's check for a correlation between people who provided gender and who did not**

In [ ]:

def assign_gender(item):
    if (item == 'male')|(item == 'female'):
        return 1
    else:
        return 0
data['gender'] = data['gender'].apply(assign_gender)

In [ ]:
test_data['gender'] = test_data['gender'].apply(assign_gender)

In [ ]:
plt.figure()
data.groupby(['msno','gender'])['is_churn'].mean().groupby(level=1).mean().plot.bar();

** we found a new dummy variable**

In [ ]:
# plotting the correlation heatmap between the variables

correl = data.corr()

mask = np.zeros_like(correl, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correl, mask=mask, cmap=cmap, 
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)






but this doesn't tell us a lot about any correlation

**Also, there is a difference in the plan listprice and actual amount paid. So surely there is a new variable in discount. **

In [ ]:
data['discount'] = data['plan_list_price'] - data['actual_amount_paid']
test_data['discount'] = test_data['plan_list_price'] - test_data['actual_amount_paid']

**See how many people churn from different cities**

In [ ]:

%matplotlib notebook
plt.figure()
data.groupby(['msno','city'])['is_churn'].mean().groupby(level=1).mean().plot.bar();
ax= plt.gca();
ax.set_xticks(np.linspace(0,22,23));


**surely, there is a huge variation in churn proportion from different cities**

Doing the same analysis for other variables such as registered_via, payment_method_id, is_churn

In [ ]:
data['registered_via'].unique()

In [ ]:
plt.figure()
data.groupby(['msno','registered_via'])['is_churn'].mean().groupby(level=1).mean().plot.bar();

In [ ]:
plt.figure()
data.groupby(['msno','payment_method_id'])['is_churn'].mean().groupby(level=1).mean().plot.bar();

In [ ]:
plt.figure()
data.groupby(['msno','is_cancel'])['is_churn'].mean().groupby(level=1).mean().plot.bar();

** now plotting a pairplot for each variable**

In [ ]:
%matplotlib inline
fig = plt.figure();
sns.pairplot(data.head(), hue='is_churn', diag_kind='kde');
fig.canvas.draw();

** now creating dummy variables for gender(whether gender priovided or not), registeres_via and city**

In [ ]:
newdf= data.join(pd.get_dummies(data['gender'])) #creating a new columns for paymenth method id dummyvariable 

gender = {} 
gender.update({True:'gender_provided'})   # create a dictionary to automatic renaming of columns
gender.update({False:'gender_not_provided'})
newdf = newdf.rename(columns=gender) #renaming the new columns
del newdf['gender']# deleting the extra columns

In [ ]:
test_newdf= test_data.join(pd.get_dummies(test_data['gender'])) #creating a new columns for paymenth method id dummyvariable 

gender = {} 
gender.update({True:'gender_provided'})   # create a dictionary to automatic renaming of columns
gender.update({False:'gender_not_provided'})
test_newdf = test_newdf.rename(columns=gender) #renaming the new columns
del test_newdf['gender']# deleting the extra columns

In [ ]:

newdf= newdf.join(pd.get_dummies(newdf['registered_via'])) #creating a new columns for paymenth method id dummyvariable 

registered_via = {} 
for i in data['registered_via'].unique(): 
    registered_via.update({i:'registered_via{}'.format(i)})   # create a dictionary to automatic renaming of columns
    

newdf = newdf.rename(columns=registered_via) #renaming the new columns
del newdf['registered_via']# deleting the extra columns

In [ ]:

test_newdf= test_newdf.join(pd.get_dummies(test_newdf['registered_via'])) #creating a new columns for paymenth method id dummyvariable 

registered_via = {} 
for i in data['registered_via'].unique(): 
    registered_via.update({i:'registered_via{}'.format(i)})   # create a dictionary to automatic renaming of columns
    

test_newdf = test_newdf.rename(columns=registered_via) #renaming the new columns
del test_newdf['registered_via']# deleting the extra columns

In [ ]:

newdf= newdf.join(pd.get_dummies(newdf['city'])) #creating a new columns for paymenth method id dummyvariable 

city = {} 
for i in data['city'].unique(): 
    city.update({i:'city{}'.format(i)})   # create a dictionary to automatic renaming of columns
    

newdf = newdf.rename(columns=city) #renaming the new columns
del newdf['city']# deleting the extra columns
newdf.head(10)

In [ ]:

test_newdf= test_newdf.join(pd.get_dummies(test_newdf['city'])) #creating a new columns for paymenth method id dummyvariable 

city = {} 
for i in data['city'].unique(): 
    city.update({i:'city{}'.format(i)})   # create a dictionary to automatic renaming of columns
    

test_newdf = test_newdf.rename(columns=city) #renaming the new columns
del test_newdf['city']# deleting the extra columns

In [ ]:

def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [ ]:
del test_data
del data

** calculating the percentage of people aged negative or more than 100** 



In [ ]:
bd_mean = np.mean(newdf['bd']);
test_bd_mean = np.mean(test_newdf['bd']);

In [ ]:
print(len(newdf[(newdf['bd']<0)|(newdf['bd']>100)])/len(newdf)*100,'%') 

In [ ]:

newdf[(newdf['bd']<0)|(newdf['bd']>100)].loc[:,'bd'] = bd_mean # filling the odd aged people with value = mean of age of users

test_newdf[(test_newdf['bd']<0)|(test_newdf['bd']>100)].loc[:,'bd'] = test_bd_mean # filling the odd aged people with value = mean of age of users


In [ ]:
test_newdf['count_of_recharge'] = 1
newdf['count_of_recharge'] = 1

In [ ]:
print(newdf.columns)

In [ ]:
print(test_newdf.columns)

In [ ]:

test_newdf_grouped = test_newdf.groupby('msno').agg({ 'bd':np.mean, 'registration_init_time':min, 
       'payment_plan_days':np.mean, 'plan_list_price':np.mean,'count_of_recharge':'sum', 'actual_amount_paid':np.mean,
       'is_auto_renew':np.mean, 'transaction_date':min, 'membership_expire_date':max,
       'is_cancel':np.mean, 'is_churn':max, 'discount':'sum',  'gender_not_provided':np.mean, 'gender_provided':np.mean,
       'registered_via3':np.mean, 'registered_via4':np.mean, 'registered_via7':np.mean,
       'registered_via9':np.mean, 'registered_via13':np.mean, 'city1':np.mean, 'city3':np.mean, 
       'city4':np.mean,'city5':np.mean, 'city6':np.mean, 'city7':np.mean, 'city8':np.mean,
       'city9':np.mean, 'city10':np.mean, 'city11':np.mean, 'city12':np.mean, 'city13':np.mean, 
       'city14':np.mean, 'city15':np.mean, 'city16':np.mean, 'city17':np.mean, 'city18':np.mean,
       'city19':np.mean, 'city20':np.mean, 'city21':np.mean, 'city22':np.mean})


In [ ]:

newdf_grouped = newdf.groupby('msno').agg({ 'bd':np.mean, 'registration_init_time':min, 
       'payment_plan_days':np.mean, 'plan_list_price':np.mean,'count_of_recharge':'sum', 'actual_amount_paid':np.mean,
       'is_auto_renew':np.mean, 'transaction_date':min, 'membership_expire_date':max,
       'is_cancel':np.mean, 'is_churn':max, 'discount':'sum', 'gender_not_provided':np.mean, 'gender_provided':np.mean,
       'registered_via3':np.mean, 'registered_via4':np.mean, 'registered_via7':np.mean,
       'registered_via9':np.mean, 'registered_via13':np.mean, 'city1':np.mean, 'city3':np.mean, 
       'city4':np.mean,'city5':np.mean, 'city6':np.mean, 'city7':np.mean, 'city8':np.mean,
       'city9':np.mean, 'city10':np.mean, 'city11':np.mean, 'city12':np.mean, 'city13':np.mean, 
       'city14':np.mean, 'city15':np.mean, 'city16':np.mean, 'city17':np.mean, 'city18':np.mean,
       'city19':np.mean, 'city20':np.mean, 'city21':np.mean, 'city22':np.mean})


In [ ]:
newdf_grouped.head(10)

In [ ]:
newdf_grouped.columns

In [ ]:

newdf_grouped[newdf_grouped.columns[-28:]] = newdf_grouped[newdf_grouped.columns[-28:]].applymap(lambda x: 1 if x>0 else 0).apply(lambda x: x.astype('int8'))
test_newdf_grouped[newdf_grouped.columns[-28:]] = test_newdf_grouped[newdf_grouped.columns[-28:]].applymap(lambda x: 1 if x>0 else 0).apply(lambda x: x.astype('int8'))# converting 0/1 for city

newdf_grouped[newdf_grouped.columns[11]].describe()

In [ ]:

newdf_grouped['discount'] = newdf_grouped['discount'].astype('int16')

In [ ]:
newdf_grouped[newdf_grouped.columns[2:5]].describe()


In [ ]:

newdf_grouped[newdf_grouped.columns[2:5]] = newdf_grouped[newdf_grouped.columns[2:5]].apply(lambda x: round(x).astype('int16'))
test_newdf_grouped[newdf_grouped.columns[2:5]] = test_newdf_grouped[newdf_grouped.columns[2:5]].apply(lambda x: round(x).astype('int16'))

In [ ]:
#churn is 6.5% which is not as bad.
np.divide(np.sum(newdf_grouped['is_churn']),newdf_grouped.index.nunique())*100

In [ ]:
np.divide(np.sum(test_newdf_grouped['is_churn']),test_newdf_grouped.index.nunique())*100

In [ ]:
newdf_grouped.head(10)


In [ ]:

newdf_grouped['days_to_buy_membership'] = newdf_grouped['transaction_date'] - newdf_grouped['registration_init_time']
newdf_grouped['tenure_membership'] = newdf_grouped['membership_expire_date']-newdf_grouped['transaction_date']
newdf_grouped['days_to_buy_membership'] = (newdf_grouped['days_to_buy_membership']/ np.timedelta64(1, 'D')).astype(int)
newdf_grouped['tenure_membership'] = (newdf_grouped['tenure_membership']/ np.timedelta64(1, 'D')).astype(int)

In [ ]:

test_newdf_grouped['days_to_buy_membership'] = test_newdf_grouped['transaction_date'] - test_newdf_grouped['registration_init_time']
test_newdf_grouped['tenure_membership'] = test_newdf_grouped['membership_expire_date']-test_newdf_grouped['transaction_date']
test_newdf_grouped['days_to_buy_membership'] = (test_newdf_grouped['days_to_buy_membership']/ np.timedelta64(1, 'D')).astype(int)
test_newdf_grouped['tenure_membership'] = (test_newdf_grouped['tenure_membership']/ np.timedelta64(1, 'D')).astype(int)

In [ ]:
newdf_grouped.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler();
scale.fit(X_train);
X_train_train = scale.transform(X_train);
X_test_trans = scale.transform(X_test);

'''
# h=change the type of these series
transactions = pd.read_csv('../input/transactions.csv', engine = 'c', sep=',')#reading the transaction file
transactions['payment_method_id'] = transactions['payment_method_id'].astype('int8')
transactions['payment_plan_days'] = transactions['payment_plan_days'].astype('int16')
transactions['plan_list_price'] = transactions['plan_list_price'].astype('int16')
transactions['actual_amount_paid'] = transactions['actual_amount_paid'].astype('int16')
transactions['is_auto_renew'] = transactions['is_auto_renew'].astype('int8') # chainging the type to boolean
transactions['is_cancel'] = transactions['is_cancel'].astype('int8')#changing the type to boolean
transactions['membership_expire_date'] = pd.to_datetime(transactions['membership_expire_date'].astype(str), infer_datetime_format = True, exact=False)
# converting the series to string and then to datetime format for easy manipulation of dates
sum(transactions.memory_usage()/1024**2) # this wouldn't change the size of df as memory occupied by object is similar to datetime
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'].astype(str), infer_datetime_format = True, exact=False)
members = pd.read_csv('../input/members_v3.csv')
members['city']=members['city'].astype('int8');
members['bd'] = members['bd'].astype('int16');
members['bd']=members['bd'].astype('int8');
members['registration_init_time'] = pd.to_datetime(members['registration_init_time'].astype(str), infer_datetime_format = True, exact=False)
#members['expiration_date'] = pd.to_datetime(members['expiration_date'].astype(str), infer_datetime_format = True, exact=False)
train = pd.read_csv('../input/train.csv')
train.head()
train['is_churn'] = train['is_churn'].astype('int8');
members_trans = members.merge(transactions, how='inner', on='msno')
data = members_trans.merge(train, how='inner', on='msno')
# deleting the previously imported df as they occupy space in memory
del transactions
del members
del train
del members_trans
def assign_gender(item):
    if (item == 'male')|(item == 'female'):
        return 1
    else:
        return 0
data['gender'] = data['gender'].apply(assign_gender)
data['discount'] = data['plan_list_price'] - data['actual_amount_paid']
newdf= data.join(pd.get_dummies(data['payment_method_id'])) #creating a new columns for paymenth method id dummyvariable 

payment_method_id = {} 
for i in data['payment_method_id'].unique():
    payment_method_id.update({i:'payment_method_id{}'.format(i)})   # create a dictionary to automatic renaming of columns

newdf = newdf.rename(columns=payment_method_id) #renaming the new columns
del newdf['payment_method_id']# deleting the extra columns
newdf.head()
newdf= newdf.join(pd.get_dummies(newdf['gender'])) #creating a new columns for paymenth method id dummyvariable 

gender = {} 
gender.update({True:'gender_provided'})   # create a dictionary to automatic renaming of columns
gender.update({False:'gender_not_provided'})
newdf = newdf.rename(columns=gender) #renaming the new columns
del newdf['gender']# deleting the extra columns
newdf.columns
newdf= newdf.join(pd.get_dummies(newdf['registered_via'])) #creating a new columns for paymenth method id dummyvariable 

registered_via = {} 
for i in data['registered_via'].unique(): 
    registered_via.update({i:'registered_via{}'.format(i)})   # create a dictionary to automatic renaming of columns
    

newdf = newdf.rename(columns=registered_via) #renaming the new columns
del newdf['registered_via']# deleting the extra columns
newdf.columns
newdf= newdf.join(pd.get_dummies(newdf['city'])) #creating a new columns for paymenth method id dummyvariable 

city = {} 
for i in data['city'].unique(): 
    city.update({i:'city{}'.format(i)})   # create a dictionary to automatic renaming of columns
    

newdf = newdf.rename(columns=city) #renaming the new columns
del newdf['city']# deleting the extra columns
newdf.head(10)
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]
bd_mean = np.mean(newdf['bd'])
newdf[(newdf['bd']<0)|(newdf['bd']>100)].loc[:,'bd'] = bd_mean # filling the odd aged people with value = mean of age of users

newdf['count_of_recharge'] = 1
newdf_grouped = newdf.groupby('msno').agg({ 'bd':np.mean, 'registration_init_time':min, 
       'payment_plan_days':np.mean, 'plan_list_price':np.mean,'count_of_recharge':'sum', 'actual_amount_paid':np.mean,
       'is_auto_renew':np.mean, 'transaction_date':min, 'membership_expire_date':max,
       'is_cancel':np.mean, 'is_churn':min, 'discount':'sum', 'payment_method_id2':np.mean,
       'payment_method_id3':sum, 'payment_method_id4':np.sum, 'payment_method_id5':np.sum,
       'payment_method_id6':np.sum, 'payment_method_id8':np.sum, 'payment_method_id10':np.sum,
       'payment_method_id11':np.sum, 'payment_method_id12':np.sum, 'payment_method_id13':np.sum,
       'payment_method_id14':np.sum, 'payment_method_id15':np.sum, 'payment_method_id16':np.sum,
       'payment_method_id17':np.sum, 'payment_method_id18':np.sum, 'payment_method_id19':np.sum,
       'payment_method_id20':np.sum, 'payment_method_id21':np.sum, 'payment_method_id22':np.sum,
       'payment_method_id23':np.sum, 'payment_method_id24':np.sum, 'payment_method_id25':np.sum,
       'payment_method_id26':np.sum, 'payment_method_id27':np.sum, 'payment_method_id28':np.sum,
       'payment_method_id29':np.sum, 'payment_method_id30':np.sum, 'payment_method_id31':np.sum,
       'payment_method_id32':np.sum, 'payment_method_id33':np.sum, 'payment_method_id34':np.sum,
       'payment_method_id35':np.sum, 'payment_method_id36':np.sum, 'payment_method_id37':np.sum,
       'payment_method_id38':np.sum, 'payment_method_id39':np.sum, 'payment_method_id40':np.sum,
       'payment_method_id41':np.sum, 'gender_not_provided':np.mean, 'gender_provided':np.mean,
       'registered_via3':np.mean, 'registered_via4':np.mean, 'registered_via7':np.mean,
       'registered_via9':np.mean, 'registered_via13':np.mean, 'city1':np.mean, 'city3':np.mean, 
       'city4':np.mean,'city5':np.mean, 'city6':np.mean, 'city7':np.mean, 'city8':np.mean,
       'city9':np.mean, 'city10':np.mean, 'city11':np.mean, 'city12':np.mean, 'city13':np.mean, 
       'city14':np.mean, 'city15':np.mean, 'city16':np.mean, 'city17':np.mean, 'city18':np.mean,
       'city19':np.mean, 'city20':np.mean, 'city21':np.mean, 'city22':np.mean})

newdf_grouped[newdf_grouped.columns[-28:]] = newdf_grouped[newdf_grouped.columns[-28:]].applymap(lambda x: 1 if x>0 else 0).apply(lambda x: x.astype('int8'))  # converting 0/1 for city
newdf_grouped[newdf_grouped.columns[12:-28]] = newdf_grouped[newdf_grouped.columns[12:-28]].apply(lambda x:x.astype('int8'))

newdf_grouped['discount'] = newdf_grouped['discount'].astype('int16')
newdf_grouped[newdf_grouped.columns[2:5]] = newdf_grouped[newdf_grouped.columns[2:5]].apply(lambda x: round(x).astype('int16'))
newdf_grouped['days_to_buy_membership'] = newdf_grouped['transaction_date'] - newdf_grouped['registration_init_time']
newdf_grouped['days_to_buy_membership'] = (newdf_grouped['days_to_buy_membership']/ np.timedelta64(1, 'D')).astype(int)'''

In [ ]:
newdf_grouped.hist('actual_amount_paid',bins= 100);

In [ ]:
#from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
#from sklearn.model_selection import train_test_split
#from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
#from sklearn.neural_network import MLPClassifier

In [ ]:
def evaluation_plots(est, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    from sklearn import metrics
    from sklearn.model_selection import learning_curve
    #model: estimator function
    y_pred=est.predict(X_test);
    print('Model R2 on train data is: ', est.score(X_train, y_train));
    print('Model R2 on test data is: ', est.score(X_test, y_test));
    print('\n\nClassification report: \n', metrics.classification_report(y_test, y_pred));
    plt.figure();
    
    sns.heatmap(metrics.confusion_matrix(y_test, y_pred), 
            cmap='viridis',annot=True, annot_kws={'fontsize':13},
            cbar=False, xticklabels=['Predicted Negative','Predicted Positive '], yticklabels=['Actual Negative','Actual Positive']);
    plt.title("Confusion Matrix", fontsize=20);
    
   # train_size, train_score, CV_score = learning_curve(est, X_train, y_train, train_sizes=np.linspace(0.1,1, 5));
    #plt.figure();
    #plt.plot(train_size, np.mean(train_score, axis=1), 'o-', color='b', label='training score');
    #plt.plot(train_size, np.mean(CV_score, axis=1), 'o-', color='r', label='CV score');
    #plt.xlabel("Training examples");
   # plt.ylabel("Score");
   # plt.fill_between(train_size, np.mean(train_score, axis=1) - np.std(train_score, axis=1),
                   #  np.mean(train_score, axis=1) + np.std(train_score, axis=1), alpha=0.1,
                  #   color="b");
   # plt.fill_between(train_size, np.mean(CV_score, axis=1) - np.std(CV_score, axis=1),
                 #    np.mean(CV_score, axis=1) + np.std(CV_score, axis=1), alpha=0.1,
                 #    color="r");
    #plt.title("Learning Curve", fontsize=20);
    #plt.legend();
   # plt.show();
    
    plt.figure();
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred);
    auc = metrics.auc(fpr, tpr);
    plt.plot(fpr, tpr, 'g-', lw=2);
    plt.plot([0,1],[0,1], 'b--', lw=3);
    #plt.gca().ylabel("True Positive rate");
    #plt.gca().xlabel("False Positive rate");
    plt.gca().text(.6,.05,"Area under the curve {}".format(np.round(auc, 2)));

In [ ]:
Logit = LogisticRegression()
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[0.01,.1,1,10,100], 'penalty':['l1','l2']}
gscv = GridSearchCV(Logit, param_grid)
gscv.fit(X_train, y_train);


In [ ]:
evaluation_plots(gscv, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test);



In [ ]:
train = newdf_grouped[newdf_grouped['membership_expire_date']<'2017-04-01']
#test = newdf_grouped[(newdf_grouped['membership_expire_date']>='2017-03-01')&(newdf_grouped['membership_expire_date']<'2017-04-01')]


In [ ]:
predictgscv.predict(X_test)

In [ ]:
print("Churn rate in february: ", sum(train['is_churn'])/len(train['is_churn']))
#print("Chaurn rate in march: ",sum(test['is_churn'])/len(test['is_churn']))

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
X_train = train[train.columns[[0,2,4,5,6,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,
              41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78]]]
y_train = train['is_churn']

In [ ]:
X_train.head()

In [ ]:
bern_X = X_train[[ 'is_auto_renew', 'city1', 'city3', 'city4',
       'city5', 'city6', 'city7', 'city8', 'city9', 'city10', 'city11',
       'city12', 'city13', 'city14', 'city15', 'city16', 'city17', 'city18',
       'city19', 'city20', 'city21', 'city22',   'gender_not_provided', 'is_cancel', 'registered_via3', 'registered_via4', 'registered_via7',
       'registered_via9', 'registered_via13']]

In [ ]:
mult_X = X_train[['count_of_recharge','payment_method_id2',
       'payment_method_id3', 'payment_method_id4', 'payment_method_id5',
       'payment_method_id6', 'payment_method_id8', 'payment_method_id10',
       'payment_method_id11', 'payment_method_id12', 'payment_method_id13',
       'payment_method_id14', 'payment_method_id15', 'payment_method_id16',
       'payment_method_id17', 'payment_method_id18', 'payment_method_id19',
       'payment_method_id20', 'payment_method_id21', 'payment_method_id22',
       'payment_method_id23', 'payment_method_id24', 'payment_method_id25',
       'payment_method_id26', 'payment_method_id27', 'payment_method_id28',
       'payment_method_id29', 'payment_method_id30', 'payment_method_id31',
       'payment_method_id32', 'payment_method_id33', 'payment_method_id34',
       'payment_method_id35', 'payment_method_id36', 'payment_method_id37',
       'payment_method_id38', 'payment_method_id39', 'payment_method_id40',
       'payment_method_id41']]


In [ ]:
gaus_X = normalize(X_train[['bd', 'payment_plan_days',  'actual_amount_paid',
        'discount', 'days_to_buy_membership']])

In [ ]:
gaus_X_test = normalize(test[['bd', 'payment_plan_days',  'actual_amount_paid',
        'discount', 'days_to_buy_membership']])
mult_X_test = test[['count_of_recharge','payment_method_id2',
       'payment_method_id3', 'payment_method_id4', 'payment_method_id5',
       'payment_method_id6', 'payment_method_id8', 'payment_method_id10',
       'payment_method_id11', 'payment_method_id12', 'payment_method_id13',
       'payment_method_id14', 'payment_method_id15', 'payment_method_id16',
       'payment_method_id17', 'payment_method_id18', 'payment_method_id19',
       'payment_method_id20', 'payment_method_id21', 'payment_method_id22',
       'payment_method_id23', 'payment_method_id24', 'payment_method_id25',
       'payment_method_id26', 'payment_method_id27', 'payment_method_id28',
       'payment_method_id29', 'payment_method_id30', 'payment_method_id31',
       'payment_method_id32', 'payment_method_id33', 'payment_method_id34',
       'payment_method_id35', 'payment_method_id36', 'payment_method_id37',
       'payment_method_id38', 'payment_method_id39', 'payment_method_id40',
       'payment_method_id41']]
bern_X_test = test[[ 'is_auto_renew', 'city1', 'city3', 'city4',
       'city5', 'city6', 'city7', 'city8', 'city9', 'city10', 'city11',
       'city12', 'city13', 'city14', 'city15', 'city16', 'city17', 'city18',
       'city19', 'city20', 'city21', 'city22',   'gender_not_provided', 'is_cancel', 'registered_via3', 'registered_via4', 'registered_via7',
       'registered_via9', 'registered_via13']]

In [ ]:
gauss.fit(gaus_X, y_train)
bern.fit(bern_X, y_train)
multi.fit(mult_X, y_train)

In [ ]:
gauss.fit(gaus_X, y_train)
bern.fit(bern_X, y_train)
multi.fit(mult_X, y_train)

In [ ]:
predict = model.predict(X_test)

In [ ]:
submission = pd.DataFrame()
submission['msno'] = newdf[(newdf['expiration_date']>='2017-04-01')&(newdf['expiration_date']<'2017-05-01')].index
submission['is_churn'] = predict
submission.set_index('msno')

In [ ]:
submission.to_csv('submission.csv')